In [1]:
import numpy as np
import torch
import os
import scipy.ndimage as ndi

def crea_roi_external_bilanciata_2d(slice_img, close_it=1, dilate_it=1, percentile=40):
    """
    Crea una ROI esterna bilanciata partendo da una slice di immagine,
    usando sogliatura percentuale, chiusura morfologica e dilatazione.
    """
    vol = slice_img.astype(np.float32).copy()
    vol[vol < 0] = 0  # Rimuove valori negativi (tipici placeholder)

    # Maschera di valori validi
    vol_valid = vol[np.isfinite(vol) & (vol > 0)]
    if vol_valid.size == 0:
        print("❌ Nessun voxel valido.")
        return np.zeros_like(slice_img, dtype=np.uint8)

    # 🟨 Soglia sui voxel intensi: percentile personalizzabile
    soglia = np.percentile(vol_valid, percentile)
    mask = vol > soglia

    # 🔄 Operazioni morfologiche
    mask = ndi.binary_closing(mask, iterations=close_it)
    mask = ndi.binary_fill_holes(mask)

    # 🧠 Selezione componente centrale o più grande
    labeled, num = ndi.label(mask)
    if num == 0:
        print("⚠️ Nessuna componente.")
        return np.zeros_like(mask, dtype=np.uint8)

    center = tuple(s // 2 for s in vol.shape)
    label_at_center = labeled[center]

    if label_at_center == 0:
        sizes = np.bincount(labeled.ravel())
        sizes[0] = 0  # ignora lo sfondo
        label_at_center = np.argmax(sizes)
        if label_at_center == 0:
            print("⚠️ Nessuna componente valida.")
            return np.zeros_like(mask, dtype=np.uint8)

    # ✅ Maschera finale con dilatazione
    mask_finale = (labeled == label_at_center)
    mask_finale = ndi.binary_dilation(mask_finale, iterations=dilate_it)

    return mask_finale.astype(np.uint8)


In [2]:
import torch
import numpy as np
import scipy.ndimage as ndi

def aggiungi_roi_external_malato_sano(data, percentile=40, close_it=1, dilate_it=1):
    vol = data["volume"]
    roi_masks = data.get("roi_masks", {})

    # Calcola EXTERNAL
    shape = vol.shape[1:]  # esclude canale
    roi_external = torch.zeros(shape, dtype=torch.uint8)

    for z in range(shape[0]):
        slice_img = vol[0, z].numpy()
        mask_ext = crea_roi_external_bilanciata_2d(
            slice_img, percentile=percentile, close_it=close_it, dilate_it=dilate_it
        )
        roi_external[z] = torch.tensor(mask_ext, dtype=torch.uint8)

    # Inserisce EXTERNAL
    roi_masks_ordinato = {"EXTERNAL": roi_external}

    # Copia le ROI esistenti
    for k, v in roi_masks.items():
        roi_masks_ordinato[k] = v

    # Crea ROI DUBBIA per ogni coppia MALATO-OP / SANO-OP
    # 6️⃣ Crea ROI DUBBIA per ogni coppia MALATO-*-OPx / SANO-OPx
    # 6️⃣ Crea ROI DUBBIA per ogni coppia MALATO-*-OPx / SANO-OPx
    keys_copy = list(roi_masks_ordinato.keys())
    for malato_key in keys_copy:
        if malato_key.startswith("MALATO-") and "-OP" in malato_key:
            op = malato_key.split("-OP")[-1]  # es. "1"
            op_key = f"OP{op}"                # es. "OP1"
            sano_key = f"SANO-{op_key}"

            if sano_key in roi_masks_ordinato:
                malato = roi_masks_ordinato[malato_key]
                sano = roi_masks_ordinato[sano_key]
                dubbia = ((malato ^ sano) > 0).to(torch.uint8)
                dubbia_key = f"DUBBIA-{op_key}"
                roi_masks_ordinato[dubbia_key] = dubbia



    # Salva aggiornato
    data["roi_masks"] = roi_masks_ordinato
    return data



In [2]:
import torch
import numpy as np
import scipy.ndimage as ndi

def aggiungi_roi_external_malato_sano(data, percentile=40, close_it=1, dilate_it=1):
    vol = data["volume"]
    roi_masks = data.get("roi_masks", {})
    shape = vol.shape[1:]

    # 1️⃣ Crea EXTERNAL
    roi_external = torch.zeros(shape, dtype=torch.uint8)
    for z in range(shape[0]):
        slice_img = vol[0, z].numpy()
        mask_ext = crea_roi_external_bilanciata_2d(
            slice_img, percentile=percentile, close_it=close_it, dilate_it=dilate_it
        )
        roi_external[z] = torch.tensor(mask_ext, dtype=torch.uint8)

    roi_masks_ordinato = {"EXTERNAL": roi_external}
    for k, v in roi_masks.items():
        roi_masks_ordinato[k] = v

    # 2️⃣ Crea MALATO_COM come intersezione tra tutte le MALATO-OPx
    malato_keys = [k for k in roi_masks_ordinato if k.startswith("MALATO-OP")]
    if malato_keys:
        malato_com = roi_masks_ordinato[malato_keys[0]].clone()
        for k in malato_keys[1:]:
            malato_com &= roi_masks_ordinato[k]
        roi_masks_ordinato["MALATO_COM"] = malato_com
    else:
        print("⚠️ Nessuna ROI MALATO-OPx trovata.")
        malato_com = torch.zeros(shape, dtype=torch.uint8)

    # 3️⃣ Crea DUBBIO_COM = (unione di SANO-OPx) - MALATO_COM
    sano_keys = [k for k in roi_masks_ordinato if k.startswith("SANO-OP")]
    if sano_keys:
        sano_union = roi_masks_ordinato[sano_keys[0]].clone()
        for k in sano_keys[1:]:
            sano_union |= roi_masks_ordinato[k]
        dubbio_com = ((sano_union > 0) & (malato_com == 0)).to(torch.uint8)
        roi_masks_ordinato["DUBBIO_COM"] = dubbio_com
    else:
        print("⚠️ Nessuna ROI SANO-OPx trovata.")

    # 4️⃣ Salva aggiornato
    data["roi_masks"] = roi_masks_ordinato
    return data


In [4]:
import ipywidgets as widgets
from IPython.display import display
from tqdm.notebook import tqdm

def processa_roi_da_widget(cartella, nome_roi_riferimento="MDC"):
    pt_files = [f for f in os.listdir(cartella) if f.endswith(".pt")]

    multi_select = widgets.SelectMultiple(
        options=pt_files,
        value=[],
        description='File:',
        layout=widgets.Layout(width='60%', height='200px')
    )

    run_button = widgets.Button(description="Applica ROI", button_style='success')
    output = widgets.Output()

    def on_click(b):
        with output:
            output.clear_output()
            print(f"📋 Inizio elaborazione di {len(multi_select.value)} file...")
            for f in tqdm(multi_select.value):
                path = os.path.join(cartella, f)
                try:
                    data = torch.load(path, map_location="cpu")

                    roi_pre = list(data.get("roi_masks", {}).keys())
                    print(f"🔍 {f} ➜ ROI prima: {roi_pre}")

                    data = aggiungi_roi_external_malato_sano(data, percentile=40, close_it=1, dilate_it=1)

                    roi_post = list(data.get("roi_masks", {}).keys())
                    print(f"✅ {f} ➜ ROI dopo:  {roi_post}")

                    torch.save(data, path)
                except Exception as e:
                    print(f"❌ Errore con {f}: {e}")

    run_button.on_click(on_click)
    display(widgets.VBox([multi_select, run_button, output]))


In [5]:
cartella_derivati = "/content/drive/MyDrive/NeuroOnco/Derivate"
processa_roi_da_widget(cartella_derivati)
